In [1]:
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from collections import Counter
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


In [2]:
def cluster_acc(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [3]:
random_seed = [1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 10000]

parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument("--dataname", default = "Quake_10x_Bladder", type = str)
parser.add_argument("--dataname", default = "data_-1.5c15", type = str)

parser.add_argument("--distribution", default = "ZINB")
parser.add_argument("--self_training", default = True)
parser.add_argument("--dims", default = [499, 256, 64, 32])
parser.add_argument("--highly_genes", default = 500)
parser.add_argument("--alpha", default = 0.001, type = float)
parser.add_argument("--gamma", default = 0.001, type = float)
parser.add_argument("--learning_rate", default = 0.0001, type = float)
parser.add_argument("--random_seed", default = random_seed)
parser.add_argument("--batch_size", default = 256, type = int)
parser.add_argument("--update_epoch", default = 10, type = int)
parser.add_argument("--pretrain_epoch", default = 1000, type = int)
parser.add_argument("--funetrain_epoch", default = 2000, type = int)
parser.add_argument("--t_alpha", default = 1.0)
parser.add_argument("--noise_sd", default = 1.5)
parser.add_argument("--error", default = 0.001, type = float)
parser.add_argument("--gpu_option", default = "0")

args = parser.parse_args("")

In [7]:
category = "real_data"
import glob2
files = glob2.glob('../real_data/*.h5')
files = [f[len("../real_data/"):-3] for f in files]
files

['10X_PBMC_select_2100',
 'mouse_ES_cell',
 'worm_neuron_cell_select_2100',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'mouse_ES_cell_select_2100',
 'mouse_bladder_cell_select_2100',
 '10X_PBMC']

In [9]:
df = pd.DataFrame(columns = ["dataset", "sczi", "run"])
for dataset in files:
    data_mat = h5py.File(f"{path}{category}/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])
    print(np.where(X ==0)[0].shape[0]/(X.shape[0]*X.shape[1]))


    X = np.ceil(X).astype(np.int)
    count_X = X
    print(X.shape, count_X.shape)
    orig_X = X.copy()
    adata = sc.AnnData(X)
    adata.obs['Group'] = Y

    X = np.ceil(X).astype(np.int)
    count_X = X
    print(X.shape, count_X.shape)

    adata = sc.AnnData(X)
    adata.obs['Group'] = Y
    adata = normalize(adata,
                      copy=True,
                      highly_genes=args.highly_genes,
                      size_factors=True,
                      normalize_input=True,
                      logtrans_input=True)
    X = adata.X.astype(np.float32)
    Y = np.array(adata.obs["Group"])
    print(X.shape, count_X.shape)

    high_variable = np.array(adata.var.highly_variable.index, dtype=np.int)
    count_X = count_X[:, high_variable]
    size_factor = np.array(adata.obs.size_factors).reshape(-1,
                                                           1).astype(np.float32)
    cluster_number = int(max(Y) - min(Y) + 1)
#         X.shape, count_X.shape
    args.dims[0]=X.shape[1]
    
    for run in range(3):
        seed = run#args.random_seed[0]
        np.random.seed(seed)
        tf.reset_default_graph()
        chencluster = autoencoder(args.dataname, args.distribution,
                                  args.self_training, args.dims, cluster_number,
                                  args.t_alpha, args.alpha, args.gamma,
                                  args.learning_rate, args.noise_sd)
        chencluster.pretrain(X, count_X, size_factor, args.batch_size,
                             args.pretrain_epoch, args.gpu_option)

        chencluster.funetrain(X, count_X, size_factor, args.batch_size,
                              args.funetrain_epoch, args.update_epoch, args.error, Y)

        kmeans_accuracy = np.around(cluster_acc(Y, chencluster.kmeans_pred), 5)
        kmeans_ARI = np.around(adjusted_rand_score(Y, chencluster.kmeans_pred), 5)
        kmeans_NMI = np.around(
            normalized_mutual_info_score(Y, chencluster.kmeans_pred), 5)
        accuracy = np.around(cluster_acc(Y, chencluster.Y_pred), 5)
        ARI = np.around(adjusted_rand_score(Y, chencluster.Y_pred), 5)
        NMI = np.around(normalized_mutual_info_score(Y, chencluster.Y_pred), 5)

        df.loc[df.shape[0]] = [dataset, ARI, run]

        df.to_pickle(f"{path}output/pickle_results/{category}_sczi.pkl")

0.922900120956327
(2100, 16653) (2100, 16653)
(2100, 16653) (2100, 16653)
(2100, 499) (2100, 16653)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

begin the pretraining





begin the funetraining
ARI 0.65355, NMI 0.71689
0 ARI 0.65355, NMI 0.71689


/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


begin the pretraining
begin the funetraining
ARI 0.517, NMI 0.67979
0 ARI 0.517, NMI 0.67979


/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


begin the pretraining
begin the funetraining
ARI 0.64967, NMI 0.71946
0 ARI 0.64967, NMI 0.71946


...

/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


begin the pretraining
begin the funetraining
ARI 0.61549, NMI 0.71626
0 ARI 0.61347, NMI 0.71594
10 ARI 0.55222, NMI 0.71056
20 ARI 0.61025, NMI 0.7247
30 ARI 0.61312, NMI 0.72535
40 ARI 0.61454, NMI 0.72682
50 ARI 0.61441, NMI 0.72699
60 ARI 0.6158, NMI 0.7281
70 ARI 0.6171, NMI 0.72849
80 ARI 0.61875, NMI 0.72984
90 ARI 0.61875, NMI 0.72984


/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


begin the pretraining
begin the funetraining
ARI 0.66697, NMI 0.73201
0 ARI 0.6673, NMI 0.7323


/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [11]:
df


,dataset,sczi,run
0,10X_PBMC_select_2100,0.65355,0
1,10X_PBMC_select_2100,0.51700,1
2,10X_PBMC_select_2100,0.64967,2
3,mouse_ES_cell,0.79592,0
4,mouse_ES_cell,0.79517,1
5,mouse_ES_cell,0.81026,2
6,worm_neuron_cell_select_2100,0.12664,0
7,worm_neuron_cell_select_2100,0.23038,1
8,worm_neuron_cell_select_2100,0.24394,2
9,worm_neuron_cell,0.07251,0


In [12]:
df.groupby("dataset").mean()

,sczi
dataset,
10X_PBMC,0.590273
10X_PBMC_select_2100,0.606740
mouse_ES_cell,0.800450
mouse_ES_cell_select_2100,0.797203
mouse_bladder_cell,0.443520
mouse_bladder_cell_select_2100,0.515573
worm_neuron_cell,0.063557
worm_neuron_cell_select_2100,0.200320
